In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from utils import utils_gn, utils_noah, utils_ivc, utils_dgrd, utils_models
import importlib
importlib.reload(utils_gn)
importlib.reload(utils_noah)
importlib.reload(utils_ivc)
importlib.reload(utils_models)
importlib.reload(utils_dgrd)
warnings.filterwarnings("ignore")

In [ ]:
# Load the test raw data
test_raw = utils_gn.read_data('test_1238.pkl')

In [ ]:
# Load saved models and transformations
cycle_model, cycle_trans = utils_gn.read_data('cycles.pkl', folder='models'), utils_gn.read_data('cycles_trans.pkl', folder='models')
cap_ir_model, cap_ir_trans = utils_gn.read_data('capacity_ir.pkl', folder='models'), utils_gn.read_data('capacity_ir_trans.pkl', folder='models')

In [ ]:
# Make predictions 
cycle_pred = cycle_model.predict(cycle_trans.transform(test_raw))
capir_pred = cap_ir_model.predict(cap_ir_trans.transform(test_raw))

In [ ]:
# Load prediction intervals
cycle_pred_interval = utils_gn.read_data('cycles_pred_interval.pkl', folder='models')
capir_pred_interval = utils_gn.read_data('capir_pred_interval.pkl', folder='models')

In [ ]:
# Create dataframe of predicted values and their prediction intervals
target_list = ['k-o', 'k-p', 'e-o', 'e-p', 'EOL', 'Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL']

prediction_df = pd.DataFrame(index=test_raw.keys(), columns=target_list)

for i, cell in enumerate(test_raw.keys()):
    prediction_df.loc[cell, ['k-o', 'k-p', 'e-o', 'e-p', 'EOL']] = cycle_pred[i]
    prediction_df.loc[cell, ['Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL']] = capir_pred[i]

for i, target in enumerate(target_list[:5]):
    prediction_df[f'{target} CI'] = cycle_pred_interval[i]

for i, target in enumerate(target_list[5:]):
    prediction_df[f'{target} CI'] = capir_pred_interval[i]

columns_rearranged = []
for target in target_list:
    columns_rearranged.extend((target, f'{target} CI'))
prediction_df = prediction_df[columns_rearranged]
prediction_df.head()

In [ ]:
# Predict the entire curves for random cells in the TEST set
import scipy.signal as sg

fig1 = plt.figure(figsize=(17, 9))
fig2 = plt.figure(figsize=(17, 9))
                          
for i, cell in enumerate(['b1c23', 'b1c42', 'b3c8', 'b3c30', 'b8c14',
                         'b1c33', 'b2c39', 'b3c0', 'b3c34', 'b8c2', 'b8c44',
                         'b2c14', 'b8c6', 'b1c39', 'b2c18', 'b8c40'
                          ]):

    # For capacity
    Q = test_raw[cell]['summary']['QDischarge']
    Q_eol = Q >= .88
    Q = Q[Q_eol]
    cl_Q = sg.medfilt(Q, 5) #utils_models.knee_elbow_detection(x_data=np.arange(len(Q)) + 1, y_data=Q, type='knee', want_clean_data=True)

    end = prediction_df.loc[cell]['EOL']
    x_Q = [1, prediction_df.loc[cell]['k-o'], prediction_df.loc[cell]['k-p'], end]
    y_Q = [cl_Q[0], prediction_df.loc[cell]['Qatk-o'], prediction_df.loc[cell]['Qatk-p'], cl_Q[-1]]

    end_ci = prediction_df.loc[cell]['EOL CI']
    ttko_ci = prediction_df.loc[cell]['k-o CI']
    ttkp_ci = prediction_df.loc[cell]['k-p CI']

   
    cb_Q = utils_models.modified_spline_evaluation(x_Q, y_Q, np.arange(int(end))+1)
    cb_Q_lw = utils_models.modified_spline_evaluation([0, ttko_ci[0], ttkp_ci[0], end_ci[0]], y_Q, np.arange(int(end_ci[0]))+1)                                     
    cb_Q_up = utils_models.modified_spline_evaluation([0, ttko_ci[1], ttkp_ci[1], end_ci[1]], y_Q, np.arange(int(end_ci[1]))+1)
                                                     
    pts_Q = np.arange(len(cl_Q))+1
    

    # For internal resistance
    ir = test_raw[cell]['summary']['IR']
    cl_ir = sg.medfilt(ir, 5) #utils_models.knee_elbow_detection(x_data=np.arange(len(ir)) + 1, y_data=ir, type='elbow', want_clean_data=True)
    
    x_ir = [1, prediction_df.loc[cell]['e-o'], prediction_df.loc[cell]['e-p'], end]
    y_ir = [cl_ir[0], prediction_df.loc[cell]['IRate-o'], prediction_df.loc[cell]['IRate-p'], prediction_df.loc[cell]['IRatEOL']]

    tteo_ci = prediction_df.loc[cell]['e-o CI']
    ttep_ci = prediction_df.loc[cell]['e-p CI']
    
    cb_ir = utils_models.modified_spline_evaluation(x_ir, y_ir, np.arange(int(end))+1)
    cb_ir_lw = utils_models.modified_spline_evaluation([0, tteo_ci[0], ttep_ci[0], end_ci[0]], y_ir, np.arange(int(end_ci[0]))+1)                                         
    cb_ir_up = utils_models.modified_spline_evaluation([0, tteo_ci[1], ttep_ci[1], end_ci[1]], y_ir, np.arange(int(end_ci[1]))+1)
                                                     
    pts_ir = np.arange(len(cl_ir))+1
    
    x1 = np.arange(0, int(end_ci[0]))+1
    x2 =  np.arange(0, int(end_ci[1]))+1
   
    
    ax1 = fig1.add_subplot(4, 4, i+1)
    ax2 = fig2.add_subplot(4, 4, i+1)

    ax1.text(0.02, 0.2, cell, transform=ax1.transAxes,
            fontsize=16, fontweight='bold', va='top')

    ax2.text(0.05, 0.95, cell, transform=ax2.transAxes,
            fontsize=16, fontweight='bold', va='top')

    ax1.plot(pts_Q, cl_Q, 'b--', label='Actual curve', linewidth=1.0)
    ax1.plot(np.arange(len(cb_Q))+1, cb_Q, color='crimson', label='Predicted curve', linewidth=2.0)
    ax1.fill(np.append(x1, x2[::-1]), np.append(cb_Q_lw, cb_Q_up[::-1]), color='crimson', label=r'90% CI', alpha=0.15)
    ax1.grid(alpha=.8)
        
    ax2.plot(pts_ir, cl_ir, 'b--', label='Actual curve', linewidth=1.0)
    ax2.plot(np.arange(len(cb_ir))+1, cb_ir, color='crimson', label='Predicted curve', linewidth=2.0)
    ax2.fill(np.append(x1, x2[::-1]), np.append(cb_ir_lw, cb_ir_up[::-1]), color='crimson', label=r'90% CI', alpha=0.15)
    ax2.grid(alpha=.8)

    if i==13:
        handles1, labels1 = ax1.get_legend_handles_labels()
        ax1.legend(handles1, labels1, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(0.8, -0.4))

        handles2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(handles2, labels2, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(0.8, -0.4))



fig1.text(0.5, 0.07, 'Cycles', ha='center', va='center', fontsize=16)
fig1.text(0.09, 0.5, 'Capacity (Ah)', ha='center', va='center', rotation='vertical', fontsize=16)

fig2.text(0.5, 0.07, 'Cycles', ha='center', va='center', fontsize=16)
fig2.text(0.08, 0.5, r'Internal Resistance ($\Omega$)', ha='center', va='center', rotation='vertical', fontsize=16)



In [ ]:
# Load train true labels and corresponding predictions in order to produce parity plots
cycles_train_labels, cycles_train_pred = utils_gn.read_data('cycles_train_labels.pkl', folder='models'), utils_gn.read_data('cycles_train_pred.pkl', folder='models')
capir_train_labels, capir_train_pred = utils_gn.read_data('capir_train_labels.pkl', folder='models'), utils_gn.read_data('capir_train_pred.pkl', folder='models')

targets_to_plot = ['k-o', 'k-p', 'e-o', 'e-p', 'EOL', 'Qatk-o', 'Qatk-p']
train_df = pd.DataFrame(columns=targets_to_plot)
train_pred_df = pd.DataFrame(columns=targets_to_plot)

for i, tg in enumerate(targets_to_plot[:-2]):
    train_df[tg] = cycles_train_labels[:, i]
    train_pred_df[tg] = cycles_train_pred[:, i]

for i, tg in enumerate(targets_to_plot[-2:]):
    train_df[tg] = capir_train_labels[:, i]
    train_pred_df[tg] = capir_train_pred[:, i]

test_df = utils_dgrd.create_knee_elbow_data(test_raw)

In [ ]:
# Generate parity plots for k-o, k-p, e-o, and e-p
fig = plt.figure(figsize=(18, 4))

for i, target in enumerate(targets_to_plot[:4]):


    ax = fig.add_subplot(1, 4, i+1)
    ax.text(0.05, 0.95, target, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top')
    
    y_train_true = train_df[target].values
    y_train_pred = train_pred_df[target].values

    y_test_true = test_df[target].values
    y_test_pred = prediction_df[target].values

  
    ax.scatter(y_train_true, y_train_pred, s=50, color='blue', alpha=0.5, label='Train')
    ax.scatter(y_test_true, y_test_pred, s=50, color='red', alpha=0.5, label='Test')
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against each other
    ax.plot(lims, lims, 'k--', alpha=0.75, zorder=100)
    ax.set_xlim(lims)
    ax.set_ylim(lims)

    if i in [0, 4]:
        ax.set_ylabel('Predicted values', fontsize=16)
 
    ax.set_xlabel('Observed Values', fontsize=16)
    
    #if i==5:
     #   handles, labels = ax.get_legend_handles_labels()
      #  ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(1.0, -0.2))
    
    # embed histogram of residuals
    subaxis = utils_models.add_sub_axes(ax, [0.6, 0.17, 0.35, 0.2])
    res_train = y_train_true - y_train_pred
    res_test = y_test_true - y_test_pred
    res = np.concatenate((res_train, res_test))
    subaxis.hist(res, bins=20, color='black', alpha=0.75, ec='black')
    subaxis.set_xlim(res.min(), -res.min())
    subaxis.set_xlabel('Residuals', fontsize=10)
    subaxis.set_ylabel('Frequency', fontsize=10)

plt.savefig(fname="plots/parity-plot-1", bbox_inches='tight')

In [ ]:
# Generate parity plots for EOL, Qatk-o, and Qatk-p
fig = plt.figure(figsize=(18, 4))

for i, target in enumerate(targets_to_plot[4:]):


    ax = fig.add_subplot(1, 4, i+1)
    ax.text(0.05, 0.95, target, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top')
    
    y_train_true = train_df[target].values
    y_train_pred = train_pred_df[target].values

    y_test_true = test_df[target].values
    y_test_pred = prediction_df[target].values

  
    ax.scatter(y_train_true, y_train_pred, s=50, color='blue', alpha=0.5, label='Train')
    ax.scatter(y_test_true, y_test_pred, s=50, color='red', alpha=0.5, label='Test')
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against each other
    ax.plot(lims, lims, 'k--', alpha=0.75, zorder=100)
    ax.set_xlim(lims)
    ax.set_ylim(lims)

    if i in [0, 4]:
        ax.set_ylabel('Predicted values', fontsize=16)
    ax.set_xlabel('Observed Values', fontsize=16)
    
    if i==1:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(0.5, -0.2))
    
    # embed histogram of residuals
    subaxis = utils_models.add_sub_axes(ax, [0.6, 0.17, 0.35, 0.2])
    res_train = y_train_true - y_train_pred
    res_test = y_test_true - y_test_pred
    res = np.concatenate((res_train, res_test))
    subaxis.hist(res, bins=20, color='black', alpha=0.75, ec='black')
    subaxis.set_xlim(res.min(), -res.min())
    subaxis.set_xlabel('Residuals', fontsize=10)
    subaxis.set_ylabel('Frequency', fontsize=10)

plt.savefig(fname="plots/parity-plot-2", bbox_inches='tight')